In [1]:
import pandas as pd
import xarray
import numpy as np

In [2]:
arr = xarray.open_dataset("rfe.nc", decode_times=False)
lluvia = arr.to_dataframe().reset_index()
lluvia.rename(columns={"T": "months", "X": "lon", "Y": "lat"}, inplace=True)
lluvia.head()

,lon,months,lat,rfe
0,-92.474998,252.5,13.525000,0.0
1,-92.474998,252.5,13.575000,0.0
2,-92.474998,252.5,13.625000,0.0
3,-92.474998,252.5,13.674999,0.0
4,-92.474998,252.5,13.724999,0.0


In [3]:
import datetime as dt
from dateutil.relativedelta import relativedelta

In [6]:
def process_date(months):
    return dt.datetime(1960, 1, 1) + relativedelta(months=int(months))

lluvia["date"] = lluvia.months.map(process_date)

In [7]:
lluvia["mes"] = lluvia.date.map(lambda x: x.month)
lluvia["anio"] = lluvia.date.map(lambda x: x.year)
lluvia.head()

,lon,months,lat,rfe,date,mes,anio
0,-92.474998,252.5,13.525000,0.0,1981-01-01,1,1981
1,-92.474998,252.5,13.575000,0.0,1981-01-01,1,1981
2,-92.474998,252.5,13.625000,0.0,1981-01-01,1,1981
3,-92.474998,252.5,13.674999,0.0,1981-01-01,1,1981
4,-92.474998,252.5,13.724999,0.0,1981-01-01,1,1981


In [8]:
lluvia = lluvia[lluvia.anio > 2001]

In [9]:
df = pd.read_csv("ndvi-completo.csv")
df.head()

,date,adm2_id,ADM2_PCODE,n_pixels,vim,vim_avg,viq,mes,anio,dia,departamento,municipio,lon,lat
0,2002-07-01,65298,GT1802,5.0,0.7442,0.7380,100.7902,7,2002,1,Izabal,Lívingston,-89.151105,15.777356
1,2002-07-11,65298,GT1802,5.0,0.7446,0.7394,100.6574,7,2002,11,Izabal,Lívingston,-89.151105,15.777356
2,2002-07-21,65298,GT1802,5.0,0.7452,0.7423,100.3636,7,2002,21,Izabal,Lívingston,-89.151105,15.777356
3,2002-08-01,65298,GT1802,5.0,0.7461,0.7456,100.0641,8,2002,1,Izabal,Lívingston,-89.151105,15.777356
4,2002-08-11,65298,GT1802,5.0,0.7476,0.7490,99.8260,8,2002,11,Izabal,Lívingston,-89.151105,15.777356


In [10]:
df = df.groupby("ADM2_PCODE").mean().reset_index()[["ADM2_PCODE", "lat", "lon"]]
df.head()

/var/folders/sg/bv36hz0d5z36h52fqpd37kn80000gn/T/ipykernel_30164/2513677182.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df = df.groupby("ADM2_PCODE").mean().reset_index()[["ADM2_PCODE", "lat", "lon"]]


,ADM2_PCODE,lat,lon
0,GT0100,14.435029,-90.548878
1,GT0101,14.635431,-90.479418
2,GT0102,14.564392,-90.465079
3,GT0103,14.554316,-90.346363
4,GT0104,14.793193,-90.372463


In [11]:
coord = lluvia[["lon", "lat"]].drop_duplicates()
coord["lluvia_code"] = range(len(coord))
coord.head()

,lon,lat,lluvia_code
25200,-92.474998,13.525000,0
25201,-92.474998,13.575000,1
25202,-92.474998,13.625000,2
25203,-92.474998,13.674999,3
25204,-92.474998,13.724999,4


In [12]:
coord.shape

(9000, 3)

In [13]:
def lluvia_code(row):
    dist = np.sqrt((coord.lon - row.lon)**2 + (coord.lat - row.lat)**2)
    return int(coord.iloc[dist.argmin()]["lluvia_code"])


df["lluvia_code"] = df.apply(lluvia_code, axis=1)
df.head()

,ADM2_PCODE,lat,lon,lluvia_code
0,GT0100,14.435029,-90.548878,3918
1,GT0101,14.635431,-90.479418,4022
2,GT0102,14.564392,-90.465079,4021
3,GT0103,14.554316,-90.346363,4321
4,GT0104,14.793193,-90.372463,4225


In [14]:
df.lluvia_code.nunique()

313

In [15]:
lluvia = lluvia.merge(coord, "left", on=["lat", "lon"])

In [16]:
lluvia.shape

(2385000, 8)

In [17]:
ndvi = pd.read_csv("ndvi-completo.csv")
ndvi.shape

(247260, 14)

In [18]:
ndvi = ndvi.merge(df[["ADM2_PCODE", "lluvia_code"]], "left", on="ADM2_PCODE")
ndvi.head()

,date,adm2_id,ADM2_PCODE,n_pixels,vim,vim_avg,viq,mes,anio,dia,departamento,municipio,lon,lat,lluvia_code
0,2002-07-01,65298,GT1802,5.0,0.7442,0.7380,100.7902,7,2002,1,Izabal,Lívingston,-89.151105,15.777356,6645
1,2002-07-11,65298,GT1802,5.0,0.7446,0.7394,100.6574,7,2002,11,Izabal,Lívingston,-89.151105,15.777356,6645
2,2002-07-21,65298,GT1802,5.0,0.7452,0.7423,100.3636,7,2002,21,Izabal,Lívingston,-89.151105,15.777356,6645
3,2002-08-01,65298,GT1802,5.0,0.7461,0.7456,100.0641,8,2002,1,Izabal,Lívingston,-89.151105,15.777356,6645
4,2002-08-11,65298,GT1802,5.0,0.7476,0.7490,99.8260,8,2002,11,Izabal,Lívingston,-89.151105,15.777356,6645


In [19]:
ndvi.shape

(247260, 15)

In [20]:
ndvi = ndvi.merge(lluvia[["lluvia_code", "mes", "anio", "rfe"]], "left", on=["lluvia_code", "mes", "anio"])

In [21]:
ndvi.rename(columns={"rfe": "precipitacion"}, inplace=True)

In [22]:
ndvi = ndvi[ndvi.anio <= 2023]
ndvi.to_csv("ndvi-lluvia.csv", index=False)